In [23]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import glob
from yaml import unsafe_load as yaml_load
import numpy as np
import tqdm
from skimage import measure
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from scipy import linalg

from keras import models

from keras_transfer_learning import dataset, model
from keras_transfer_learning.data import compare

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


TODOs:

* read https://arxiv.org/pdf/1805.12462.pdf

In [3]:
print('\n'.join(sorted([f.rpartition(os.path.sep)[-1] for f in glob.glob(os.path.join('.', 'models', '*'))])))

A00_unet_stardist_hl60-high-noise_P_002
A00_unet_stardist_hl60-high-noise_P_005
A00_unet_stardist_hl60-high-noise_P_010
A00_unet_stardist_hl60-high-noise_P_050
A00_unet_stardist_hl60-high-noise_P_200
A00_unet_stardist_hl60-high-noise_R_002
A00_unet_stardist_hl60-high-noise_R_005
A00_unet_stardist_hl60-high-noise_R_010
A00_unet_stardist_hl60-high-noise_R_050
A00_unet_stardist_hl60-high-noise_R_200
A00_unet_stardist_hl60-high-noise_R_F
A00_unet_stardist_hl60-low-noise_P_002
A00_unet_stardist_hl60-low-noise_P_005
A00_unet_stardist_hl60-low-noise_P_010
A00_unet_stardist_hl60-low-noise_P_050
A00_unet_stardist_hl60-low-noise_P_200
A00_unet_stardist_hl60-low-noise_R_002
A00_unet_stardist_hl60-low-noise_R_005
A00_unet_stardist_hl60-low-noise_R_010
A00_unet_stardist_hl60-low-noise_R_050
A00_unet_stardist_hl60-low-noise_R_200
A00_unet_stardist_hl60-low-noise_R_F
A01_unet_stardist_hl60-high-noise_P_002
A01_unet_stardist_hl60-high-noise_P_005
A01_unet_stardist_hl60-high-noise_P_010
A01_unet_stardi

In [4]:
model_name = 'A00_unet_stardist_hl60-high-noise_R_F'

In [5]:
model_dir = os.path.join('.', 'models', model_name)
m = model.Model(model_dir=model_dir, load_weights='last')

Instructions for updating:
Colocations handled automatically by placer.


## Create the feature model

In [6]:
def model_up_to_layer(keras_model, layer_name):
    for l in keras_model.layers:
        if l.name == layer_name:
            break
    return models.Model(keras_model.input, l.output)

feature_model = model_up_to_layer(m.model, 'activation_1')
features_size = feature_model.output.shape[-1].value

## Compute the mean and var for model data

In [7]:
d = dataset.Dataset(m.config)
x_test, _ = d.create_test_dataset()

In [21]:
def get_mgf(data, limit=20):
    if limit is not None:
        data = [data[i] for i in np.random.permutation(len(data))[:limit]]
    
    features_list = []
    for img in tqdm.tqdm(data):
        pred = feature_model.predict(img[None, ..., None])
        features_list.append(np.reshape(pred, (-1, features_size)))

    features = np.concatenate(features_list)
    mean = np.mean(features, axis=0)
    cov = np.cov(features, rowvar=0)
    return mean, cov

In [11]:
mean, cov = get_mgf(x_test)

100%|██████████| 20/20 [00:45<00:00,  2.29s/it]


In [30]:
mean_1, cov_1 = get_mgf(x_test)


100%|██████████| 20/20 [00:29<00:00,  1.39s/it]

## Compute the mean and var for other data

In [17]:
with open(os.path.join('configs', 'data', 'dsb2018.yaml'), 'r') as f:
    new_data_conf = { 'data': yaml_load(f) }
new_d = dataset.Dataset(new_data_conf)
new_x_test, _ = new_d.create_test_dataset()

In [22]:
new_mean, new_cov = get_mgf(new_x_test)


100%|██████████| 20/20 [00:51<00:00,  2.45s/it]

In [32]:
new_mean_1, new_cov_1 = get_mgf(new_x_test)


100%|██████████| 20/20 [00:32<00:00,  1.09s/it]

## Compute the distance

In [27]:
def frechet_distance(mean_1, cov_1, mean_2, cov_2):
    return np.sum(np.square(mean_1 - mean_2)) + \
            np.trace(cov_1 + cov_2 - 2 * linalg.fractional_matrix_power(np.dot(cov_1, cov_2), 1/2))

In [28]:
frechet_distance(mean, cov, new_mean, new_cov)

5368.415027869489

In [31]:
frechet_distance(mean, cov, mean_1, cov_1)

1.0572058893706213

In [33]:
frechet_distance(new_mean, new_cov, new_mean_1, new_cov_1)

(273.62129750548485-3.0255722957704123e-07j)